# 一、输入输出基础

最简单直接的输入来自键盘操作，比如下面这个例子

In [1]:
name = input('your name:')
gender = input('you are a boy?(y/n)')

your name: darrenzhang
you are a boy?(y/n) y


In [2]:
welcome_str = 'Welcome to the matrix {prefix} {name}.'
welcome_dic = {
    'prefix': "Mr." if gender == 'y' else 'Mrs',
    'name': name
}

print('authorizing...')
print(welcome_str.format(**welcome_dic))

authorizing...
Welcome to the matrix Mr. darrenzhang.


`input()` 函数暂停程序运行，同时等待键盘输入；直到回车被按下，函数的参数即为提示语，**输入的类型永远是字符串型（str）**。

print() 函数则接受字符串、数字、字典、列表甚至一些自定义类的输出。

In [4]:
a = input("please input 'a'：")
b = input("please input 'b'：")

print('a + b = {}'.format(a + b))  # 由于input() 的类型为字符串，所以 a+b 为字符串的拼接
print('type of a is {}, type of b is {}'.format(type(a), type(b)))
print('a + b = {}'.format(int(a) + int(b)))

please input 'a'： 23
please input 'b'： 23


a + b = 2323
type of a is <class 'str'>, type of b is <class 'str'>
a + b = 46


> 注意，把 `str` 强制转换为 `int` 请用 `int()`，转为浮点数请用 `float()`。而在生产环境中使用强制转换时，请记得加上 try except（即错误和异常处理）

`Python` 对 `int类型` 没有最大限制（相比之下， `C++` 的 `int` 最大为 `2147483647`，超过这个数字会产生溢出），但是对 float 类型依然有精度限制。

这种特点，平常编码过程中一定要时刻提防，避免因为对边界条件判断不清而造成 `bug` 甚至 `0day（危重安全漏洞）`。

> 我们回望一下币圈。2018 年 4 月 23 日中午 11 点 30 分左右，BEC 代币智能合约被黑客攻击。黑客**利用数据溢出的漏洞**，攻击与美图合作的公司美链 BEC 的智能合约，成功地向两个地址转出了天量级别的 BEC 代币，导致市场上的海量 BEC 被抛售，该数字货币的价值也几近归零，给 BEC 市场交易带来了毁灭性的打击。

由此可见，虽然输入输出和类型处理事情简单，但我们一定要慎之又慎。毕竟相当比例的安全漏洞，都来自随意的 I/O 处理。

# 二、文件输入输出

命令行的输入输出，只是 Python 交互的最基本方式，适用一些简单小程序的交互。**而生产级别的 Python 代码，大部分 I/O 则来自于文件、网络、其他进程的消息等等**。接下来，我们来详细分析一个文本文件读写。

在media下面有个in.txt文件，下面对其进行文件操作：
- 读取文件；
- 去除所有标点符号和换行符，并把所有大写变成小写；
- 合并相同的词，统计每个词出现的频率，并按照词频从大到小排序；
- 将结果按行输出到文件 out.txt。

In [5]:
import re

def parse_txt(text):
    # 使用正则表达式取出标点符号和换行符
    text = re.sub(r'[^\w]', ' ', text)
    
    # 转化为小写
    text = text.lower()
    
    # 生成所有单词的列表
    word_list = text.split(' ')
    
    # 去除空白单词
    word_list = filter(None, word_list)
    
    # 生成单词和词频的词典
    word_cnt = {}
    for word in word_list:
        if word not in word_cnt:
            word_cnt[word] = 0
        word_cnt[word] += 1
        
    # 按照词频进行排序
    sorted_word_cnt = sorted(word_cnt.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_word_cnt


In [7]:
# 读取文件
with open('./media/in.txt', 'r') as f:
    test = f.read()

word_and_freq = parse_txt(test)

In [8]:
# 写入文件
with open('out.put', 'w') as f:
    for word, freq in word_and_freq:
        f.write('{} {} \n'.format(word, freq))

我们先要用 `open()函数` 拿到文件的指针。
- 第一个参数指定文件位置（相对位置或者绝对位置）；
- 第二个参数，表示读写模式
    - 如果是 `r` 表示读取，
    - 如果是 `w` 则表示写入，
    - 当然也可以用 `rw` ，表示读写都要。
    - `a` 则是一个不太常用（但也很有用）的参数，表示追加（append），这样打开的文件，如果需要写入，会从原始文件的最末尾开始写入。

在日常工作中，代码权限管理非常重要。如果你只需要读取文件，就不要请求写入权限。这样在某种程度上可以降低 bug 对整个系统带来的风险。

在拿到指针后，我们可以通过 `read()函数`，来读取文件的全部内容。代码 `text = f.read()` ，即表示把文件所有内容读取到内存中，并赋值给变量 text。这么做自然也是有利有弊：
- 优点是方便，接下来我们可以很方便地调用 parse 函数进行分析；
- 缺点是如果文件过大，一次性读取可能造成内存崩溃。

这时，我们可以给 `read()` 指定参数 `size` ，用来表示读取的最大长度。还可以通过 `readline()函数`，每次读取一行，这种做法常用于数据挖掘（Data Mining）中的数据清洗，在写一些小的程序时非常轻便。如果每行之间没有关联，这种做法也可以降低内存的压力。而 `write()函数`，可以把参数中的字符串输出到文件中，也很容易理解。

如果使用 `open()函数` 来进行文件读写操作，`open()函数` 对应于 `close()函数`，如果打开了文件，在完成读取任务后，就应该立刻关掉它。

而如果你使用了 `with` 语句，就不需要显式调用 `close()`。在 with 的语境下任务执行完毕后，close() 函数会被自动调用，代码也简洁很多。

最后需要注意的是，所有 I/O 都应该进行错误处理。因为 I/O 操作可能会有各种各样的情况出现，而一个健壮（robust）的程序，需要能应对各种情况的发生，而不应该崩溃（故意设计的情况除外）

# 三、JSON 序列化与实战

JSON（JavaScript Object Notation）是一种轻量级的数据交换格式，它的设计意图是**把所有事情都用设计的字符串来表示**，这样既方便在互联网上传递信息，也方便人进行阅读（相比一些 binary 的协议）。JSON 在当今互联网中应用非常广泛，也是每一个用 Python 程序员应当熟练掌握的技能点。

设想一个情景，你要向交易所购买一定数额的股票。那么，你需要提交股票代码、方向（买入 / 卖出）、订单类型（市价 / 限价）、价格（如果是限价单）、数量等一系列参数，而这些数据里，有字符串，有整数，有浮点数，甚至还有布尔型变量，全部混在一起并不方便交易所解包。那该怎么办呢？

其实，我们要讲的 `JSON` 正能解决这个场景。你可以把它简单地理解为两种黑箱：第一种，输入这些杂七杂八的信息，比如 Python 字典，输出一个字符串；第二种，输入这个字符串，可以输出包含原始信息的 Python 字典。具体代码如下：

In [10]:
import json

params = {
    'symbol': '123456',
    'type': 'limit',
    'price': 123.4,
    'amount': 23
}

params_str = json.dumps(params)  # 接受python的基本数据类型，然后将其序列化为string类型

print('after json serialization')
print('type of params_str = {}, params_str = {}'.format(type(params_str), params))

original_params = json.loads(params_str)  # 接受一个合法的字符串，然后将其反序列化为python的基本数据类型


print('after json deserialization')
print('type of original_params = {}, original_params = {}'.format(type(original_params), params))


after json serialization
type of params_str = <class 'str'>, params_str = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}
after json deserialization
type of original_params = <class 'dict'>, original_params = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}


- `json.dumps()` 这个函数，接受 Python 的基本数据类型，然后将其序列化为 `string`；
- `json.loads()` 这个函数，接受一个合法字符串，然后将其反序列化为 `Python` 的基本数据类型。

不过在日常编码过程中，请记得加上异常处理。不然，哪怕只是给 `json.loads()` 发送了一个非法字符串，而你没有 `catch` 到，程序就会崩溃了。

到这一步，你可能会想，如果我要输出字符串到文件，或者从文件中读取 `JSON字符串`，又该怎么办呢？是的，你仍然可以使用上面提到的 `open()` 和 `read()/write()` ，先将字符串读取/输出到内存，再进行 `JSON 编码/解码`，当然这有点麻烦。

In [12]:
import json

params = {
    'symbol': '123456',
    'type': 'limit',
    'price': 123.4,
    'amount': 23
}

with open('params.json', 'w') as f:
    params_str = json.dump(params, f)

with open('params.json', 'r') as f:
    original_params = json.load(f)

print('after json deserialization')
print('type of original_params = {}, original_params = {}'.format(type(original_params), original_params))

after json deserialization
type of original_params = <class 'dict'>, original_params = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}


这样，我们就简单清晰地实现了读写 `JSON` 字符串的过程。当开发一个第三方应用程序时，你可以通过 `JSON` 将用户的个人配置输出到文件，方便下次程序启动时自动读取。这也是现在普遍运用的成熟做法。

那么 `JSON` 不仅仅是唯一的选择，它只是轻量级应用中最方便的选择之一。`Google`，开源了一个叫做 `Protocol Buffer`，相比于 `JSON`，它的优点是**生成优化后的二进制文件，因此性能更好**。但与此同时，生成的二进制序列，是不能直接阅读的。它在 `TensorFlow` 等很多对性能有要求的系统中都有广泛的应用。

# 总结
这节课，我们主要学习了 Python 的 `普通 I/O` 和 `文件 I/O`，同时了解了 `JSON序列化` 的基本知识，并通过具体的例子进一步掌握。

再次强调一下需要注意的几点：
- `I/O` 操作需谨慎，一定要进行充分的错误处理，并细心编码，防止出现编码漏洞；
- 编码时，对内存占用和磁盘占用要有充分的估计，这样在出错时可以更容易找到原因；
- `JSON` 序列化是很方便的工具，要结合实战多多练习；
- 代码尽量简洁、清晰，写代码也要有仪式感。

# 思考题 

- 第一问：你能否把 NLP 例子中的 word count 实现一遍？不过这次，in.txt 可能非常非常大（意味着你不能一次读取到内存中），而 output.txt 不会很大（意味着重复的单词数量很多）。提示：你可能需要每次读取一定长度的字符串，进行处理，然后再读取下一次的。但是如果单纯按照长度划分，你可能会把一个单词隔断开，所以需要细心处理这种边界情况。

In [13]:
def parse_readline(infile):
    # 生成单词和词频的字典
    word_cnt = {}
    while True:
        text = infile.readline()
        if not text:
            raise EOFError("no this text")
            break
        # 去除标点和换行
        text = re.sub(r'[^\w]', ' ', text)
        # 转换为小写
        text = text.lower()
        # 单词列表
        word_list = text.split(' ')
        # 去除空白单词
        word_list = filter(None, word_list)
        
        for word in word_list:
            if word not in word_cnt:
                word_cnt[word] = 0
            word_cnt[word] += 1
        
        # 按照词频排序
        sorted_word_cnt = sorted(word_cnt.items(), key = lambda x:x[1], reverse=True)
        return sorted_word_cnt

In [16]:
with open("media/in.txt", "r") as f:
    word_and_freq = parse_readline(f)
    print(word_and_freq)

[('a', 3), ('i', 2), ('have', 2), ('dream', 2), ('will', 2), ('by', 2), ('the', 2), ('of', 2), ('their', 2), ('that', 1), ('my', 1), ('four', 1), ('little', 1), ('children', 1), ('one', 1), ('day', 1), ('live', 1), ('in', 1), ('nation', 1), ('where', 1), ('they', 1), ('not', 1), ('be', 1), ('judged', 1), ('color', 1), ('skin', 1), ('but', 1), ('content', 1), ('character', 1), ('today', 1)]


In [17]:
with open("out_readline.txt", "w") as f:
    for word, freq in word_and_freq:
        f.write('{} {} \n'.format(word, freq))

- 第二问：你应该使用过类似百度网盘、Dropbox 等网盘，但是它们可能空间有限（比如 5GB）。如果有一天，你计划把家里的 100GB 数据传送到公司，可惜你没带 U 盘，于是你想了一个主意：每次从家里向 Dropbox 网盘写入不超过 5GB 的数据，而公司电脑一旦侦测到新数据，就立即拷贝到本地，然后删除网盘上的数据。等家里电脑侦测到本次数据全部传入公司电脑后，再进行下一次写入，直到所有数据都传输过去。根据这个想法，你计划在家写一个 server.py，在公司写一个 client.py 来实现这个需求。

提示：我们假设每个文件都不超过 5GB。你可以通过写入一个控制文件（config.json）来同步状态。

不过，要小心设计状态，这里有可能产生 `race condition`。你也可以通过直接侦测文件是否产生，或者是否被删除来同步状态，这是最简单的做法。